In [1]:
# Configurar el path para importar los módulos
import sys
import os

# Añadir el directorio raíz del proyecto al path
root_dir = os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
if root_dir not in sys.path:
    sys.path.append(root_dir)

In [ ]:
# Importar las funciones necesarias
from db.conexion_db import crear_conexion, cerrar_conexion
from db.querys_db import insert_data, consult_data,procedure_data,procedure_data_param
from abastecimiento import reservas_avisorden_ordenes
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [3]:
connection = crear_conexion()
procedure_data(connection,"abastece_plan_reservas")
procedure_data(connection,"abastece_plan_avisorden")
procedure_data(connection,"abastece_plan_ordenes")
procedure_data(connection,'abastece_plan_flota')
procedure_data_param(connection, "abastece_plan_pedidospendiente", ('2024-11-01',))   
cerrar_conexion(connection)

Conexion Exitosa
El procedimiento se ah ejecutado correctamente
El procedimiento se ah ejecutado correctamente
El procedimiento se ah ejecutado correctamente
El procedimiento se ah ejecutado correctamente
El procedimiento se ha ejecutado correctamente
conexion cerrada


In [4]:
connection = crear_conexion()
procedure_data(connection,"abastece_plan_inv_mb52")       
cerrar_conexion(connection)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Conexion Exitosa
El procedimiento se ah ejecutado correctamente
conexion cerrada


In [5]:
connection = crear_conexion()
reservas = reservas_avisorden_ordenes(connection)
cerrar_conexion(connection)

Conexion Exitosa
conexion cerrada


In [6]:
connection = crear_conexion()
flota = consult_data(connection,f"select equipo,linegnombre from plan_flota")
cerrar_conexion(connection)

Conexion Exitosa
conexion cerrada


In [7]:
connection = crear_conexion()
estado_pedidosxpendiente = consult_data(connection,f"select * from plan_pedidospendientes_resumen")
cerrar_conexion(connection)

Conexion Exitosa
conexion cerrada


In [8]:
pl_inventarioMB52 = "plan_inventario_resumen"
connection = crear_conexion()
planinventarioresumen = consult_data(connection,f"select material,centro,total from {pl_inventarioMB52}")
cerrar_conexion(connection)

Conexion Exitosa
conexion cerrada


In [9]:
tbl_inventario_resumen = pd.DataFrame(planinventarioresumen)
tbl_flota = pd.DataFrame(flota)
tbl_estado_pedidosxpendiente = pd.DataFrame(estado_pedidosxpendiente)

In [10]:

tbl_estado_pedidosxpendiente['identificador'] = tbl_estado_pedidosxpendiente['doc_compra'] + tbl_estado_pedidosxpendiente['pos']

In [11]:
filtrar_cabezeras_estadoxpedidos = tbl_estado_pedidosxpendiente[['identificador','doc_compra','material','centro','por_entr_ctd','fecha_entrega','nombre_proveedor']].copy()
tbl_inventario_resumen['total'] = tbl_inventario_resumen['total'].astype(float)
filtrar_cabezeras_estadoxpedidos['por_entr_ctd'] = filtrar_cabezeras_estadoxpedidos['por_entr_ctd'].astype(float)
filtrar_cabezeras_estadoxpedidos['fecha_entrega'] = pd.to_datetime(filtrar_cabezeras_estadoxpedidos['fecha_entrega'])
filtrar_cabezeras_estadoxpedidos['nombre_proveedor'] = filtrar_cabezeras_estadoxpedidos['nombre_proveedor'].str[0:10]

In [12]:
reservasCxC = reservas.loc[(reservas['ate_accion']=='MERCADERIA') & (reservas['ctd_dif'] > 0) ]
filtrar_cabezeras_reservasCxC = reservasCxC[['index','identificador','material','centro_x','ctd_dif','stock_asignado','trasl_ctd_C154','trasl_ctd_C040',
                                             'trasl_ctd_C080','trasl_ctd_C200','trasl_ctd_C152','trasl_fecha','Ped_nro','Ped_Fecha','Ped_ctd','Ped_proveedor']].copy()

In [13]:
filtrar_cabezeras_estadoxpedidos.columns = filtrar_cabezeras_estadoxpedidos.columns.str.lower()
filtrar_cabezeras_reservasCxC.rename(columns={'centro_x' : 'centro'},inplace=True)

In [14]:
filtrar_cabezeras_estadoxpedidos.loc[:,'material'] = filtrar_cabezeras_estadoxpedidos.loc[:,'material'].str.replace(' ','')
tbl_inventario_resumen.loc[:,'material'] = tbl_inventario_resumen.loc[:,'material'].str.replace(' ','')
filtrar_cabezeras_reservasCxC.loc[:,'material'] = filtrar_cabezeras_reservasCxC.loc[:,'material'].str.replace(' ','')

In [15]:
tbl_inventario_resumen['identificador'] = tbl_inventario_resumen['material'] + tbl_inventario_resumen['centro']

In [16]:
def _asign_stock_centros(reservas_grupo, inventario_grupo):
    # Obtener el stock disponible para el grupo
    stock_disponible = inventario_grupo['total'].iloc[0]
    
    # Inicializar la cantidad asignada
    cantidad_asignada = []
    stock_restante = stock_disponible
    
    # Calcular la cantidad asignada para cada reserva
    for ctd_dif in reservas_grupo['ctd_dif']:
        if stock_restante <= 0:
            # Si no hay stock disponible, asignar 0
            cantidad_asignada.append(0.0)
        else:
            # Asignar la cantidad mínima entre lo pedido y el stock restante
            asignacion = min(ctd_dif, stock_restante)
            cantidad_asignada.append(asignacion)
            stock_restante -= asignacion
    
    # Asignar la cantidad ajustada usando .loc para evitar SettingWithCopyWarning
    reservas_grupo.loc[:, 'stock_asignado'] = cantidad_asignada
    
    # Actualizar el inventario restante
    inventario_grupo.loc[:, 'total'] = stock_restante
    
    return reservas_grupo, inventario_grupo

In [17]:
def asignar_stock_centroxcentro(reservas, inventario):
    # Asegurarse de que las reservas estén ordenadas por la columna 'index'
    reservas = reservas.sort_values(by='index')
    
    # Listas para almacenar resultados
    reservas_actualizadas = []
    inventario_actualizado = []
    
    # Agrupar por 'centro' y 'material'
    grupos = reservas.groupby(['centro', 'material'])
    
    for (centro, material), reservas_grupo in grupos:
        # Filtrar el inventario para el grupo actual
        inventario_grupo = inventario[(inventario['centro'] == centro) & (inventario['material'] == material)]
        
        if not inventario_grupo.empty:
            # Procesar el grupo
            reservas_procesadas, inventario_procesado = _asign_stock_centros(reservas_grupo, inventario_grupo)
            reservas_actualizadas.append(reservas_procesadas)
            inventario_actualizado.append(inventario_procesado)
        else:
            # Si no hay inventario, mantener las reservas sin cambios
            reservas_actualizadas.append(reservas_grupo)
    
    # Concatenar resultados
    reservas_final = pd.concat(reservas_actualizadas, ignore_index=True)
    inventario_final = pd.concat(inventario_actualizado, ignore_index=True)
    
    return reservas_final, inventario_final

In [18]:
pedido_modif, inv_modif = asignar_stock_centroxcentro(filtrar_cabezeras_reservasCxC,tbl_inventario_resumen)

In [19]:
obtener_stockasignado = pedido_modif[['identificador','stock_asignado']]
obtener_stock_modificado = inv_modif[['identificador','total']]

In [20]:
modificar_stockasignado_reservas = filtrar_cabezeras_reservasCxC.merge(obtener_stockasignado,how='left',on='identificador')
modificar_stock_inventario = tbl_inventario_resumen.merge(obtener_stock_modificado,how='left',on='identificador')

In [21]:
modificar_stockasignado_reservas.drop(columns='stock_asignado_x', inplace=True)
modificar_stockasignado_reservas.rename(columns={'stock_asignado_y' : 'stock_asignado'}, inplace=True)

In [22]:
modificar_stockasignado_reservas['diferencia'] = modificar_stockasignado_reservas['ctd_dif'] - modificar_stockasignado_reservas['stock_asignado']
reservas_pendientes = modificar_stockasignado_reservas.loc[modificar_stockasignado_reservas['diferencia'] >0].copy()

reservas_pendientes.drop(columns='diferencia', inplace=True)
modificar_stockasignado_reservas.drop(columns='diferencia', inplace=True)


In [23]:
reservas_pendientes['total'] = reservas_pendientes['stock_asignado']
modificar_stockasignado_reservas['total'] = modificar_stockasignado_reservas['stock_asignado']

In [24]:
modificar_stock_inventario.loc[modificar_stock_inventario['total_y'].isna(), 'total_y'] = modificar_stock_inventario['total_x']
modificar_stock_inventario['total'] = modificar_stock_inventario['total_y']
modificar_stock_inventario.drop(columns={'total_x','total_y'}, inplace= True)

In [25]:
stock_restante = modificar_stock_inventario.loc[modificar_stock_inventario['total'] > 0].copy()

In [26]:
def _asign_stock_restante(_reservas, _inventario): 
    reservas = _reservas.copy()
    inventario = _inventario.copy()
    for idx, row in reservas.iterrows():
        material = row['material']
        stock_xasignado = row['total']
        ctd_dif = row['ctd_dif']
        diferencia = ctd_dif - stock_xasignado
        if diferencia > 0:
            stock_disponible = inventario.loc[(inventario['material'] == material),'total'].values[0]
            centro = inventario.loc[inventario['material'] == material,'centro'].values[0]
            cantidad_ajustada = min(diferencia,stock_disponible)
            reservas.at[idx,f'trasl_ctd_{centro}'] = cantidad_ajustada
            reservas.at[idx,f'total'] = stock_xasignado +cantidad_ajustada
            inventario.loc[(inventario['material'] == material) & (inventario['centro'] == centro), 'total'] -=cantidad_ajustada

    return [reservas, inventario]

In [27]:
def _asignacion_centro_restante(reservas,inventarios,centro):

    reserva = reservas.copy()
    inventario = inventarios.copy()
    inv_fil_cen = inventario[(inventario['centro'] == centro)]
    materiales = pd.DataFrame(inv_fil_cen['material'].unique())
    materiales.columns = ['material']
    concatenar_reservas = []
    concatenar_inventario = []

    for _,material in materiales.iterrows():
        pedido_modi = reserva[(reserva['material'] == material['material'])]
        centros_filtrados = inv_fil_cen[(inv_fil_cen['material'] == material['material'])]
        _reservas, _inventario = _asign_stock_restante(pedido_modi,centros_filtrados)
        concatenar_reservas.append(_reservas)
        concatenar_inventario.append(_inventario)        
    pedidos_consolidados = pd.concat(concatenar_reservas)
    inventario_reducido = pd.concat(concatenar_inventario)
    
    return pedidos_consolidados, inventario_reducido

In [28]:
pedido_modif2, inventario_modif2 = _asignacion_centro_restante(reservas_pendientes,stock_restante,'C154')

In [29]:
fil_pedido_modif2 = pedido_modif2[['identificador','trasl_ctd_C154','total']]
fil_inventario_modif2 = inventario_modif2[['identificador','total']]

In [30]:
agregar_stockc154xreservas = modificar_stockasignado_reservas.merge(fil_pedido_modif2, how='left', on='identificador')
modificar_stockc154xinventario = modificar_stock_inventario.merge(fil_inventario_modif2, how='left', on='identificador')

In [31]:
agregar_stockc154xreservas.loc[agregar_stockc154xreservas['trasl_ctd_C154_y'].isna(), 'trasl_ctd_C154_y'] = 0
agregar_stockc154xreservas.loc[agregar_stockc154xreservas['total_y'].isna(), 'total_y'] = agregar_stockc154xreservas['total_x']

modificar_stockc154xinventario.loc[modificar_stockc154xinventario['total_y'].isna(), 'total_y'] = modificar_stockc154xinventario['total_x']

In [32]:
agregar_stockc154xreservas['trasl_ctd_C154'] = agregar_stockc154xreservas['trasl_ctd_C154_y']
agregar_stockc154xreservas['total'] = agregar_stockc154xreservas['total_y']

agregar_stockc154xreservas.drop(columns=['trasl_ctd_C154_y','trasl_ctd_C154_x'], inplace=True)
agregar_stockc154xreservas.drop(columns=['total_y','total_x'], inplace=True)


modificar_stockc154xinventario['total'] = modificar_stockc154xinventario['total_y']
modificar_stockc154xinventario.drop(columns=['total_x','total_y'], inplace=True)


In [33]:
agregar_stockc154xreservas['diferencia'] = agregar_stockc154xreservas['ctd_dif'] - agregar_stockc154xreservas['total']
reservas_pendientes2 = agregar_stockc154xreservas.loc[agregar_stockc154xreservas['diferencia'] > 0].copy()

agregar_stockc154xreservas.drop(columns='diferencia', inplace=True)
reservas_pendientes2.drop(columns='diferencia', inplace=True)


In [34]:
stock_restante2 = modificar_stockc154xinventario.loc[modificar_stockc154xinventario['total'] > 0].copy()

In [35]:
pedido_modif3, inventario_modif3 = _asignacion_centro_restante(reservas_pendientes2,stock_restante2,'C152')

In [36]:
fil_pedido_modif3 = pedido_modif3[['identificador','trasl_ctd_C152','total']]
fil_inventario_modif3 = inventario_modif3[['identificador','total']]

In [37]:
agregar_stockc152reservas = agregar_stockc154xreservas.merge(fil_pedido_modif3,how='left', on='identificador')
modificar_stockc152xinventario = modificar_stockc154xinventario.merge(fil_inventario_modif3, how='left', on='identificador')

In [38]:
agregar_stockc152reservas.loc[agregar_stockc152reservas['trasl_ctd_C152_y'].isna(), 'trasl_ctd_C152_y'] = 0
agregar_stockc152reservas.loc[agregar_stockc152reservas['total_y'].isna(), 'total_y'] = agregar_stockc152reservas['total_x']

modificar_stockc152xinventario.loc[modificar_stockc152xinventario['total_y'].isna(), 'total_y'] = modificar_stockc152xinventario['total_x']

In [39]:
agregar_stockc152reservas['trasl_ctd_C152'] = agregar_stockc152reservas['trasl_ctd_C152_y']
agregar_stockc152reservas['total'] = agregar_stockc152reservas['total_y']

agregar_stockc152reservas.drop(columns=['trasl_ctd_C152_y','trasl_ctd_C152_x'], inplace=True)
agregar_stockc152reservas.drop(columns=['total_y','total_x'], inplace=True)


modificar_stockc152xinventario['total'] = modificar_stockc152xinventario['total_y']
modificar_stockc152xinventario.drop(columns=['total_x','total_y'], inplace=True)


In [40]:
agregar_stockc152reservas['diferencia'] = agregar_stockc152reservas['ctd_dif'] - agregar_stockc152reservas['total']
reservas_pendientes3 = agregar_stockc152reservas.loc[agregar_stockc152reservas['diferencia'] > 0].copy()

agregar_stockc152reservas.drop(columns='diferencia', inplace=True)
reservas_pendientes3.drop(columns='diferencia', inplace=True)


In [41]:
stock_restante3 = modificar_stockc152xinventario.loc[modificar_stockc152xinventario['total'] > 0].copy()

In [42]:
pedido_modif4, inventario_modif4 = _asignacion_centro_restante(reservas_pendientes3,stock_restante3,'C200')

In [43]:
fil_pedido_modif4 = pedido_modif4[['identificador','trasl_ctd_C200','total']]
fil_inventario_modif4 = inventario_modif4[['identificador','total']]

In [44]:
agregar_stockc200reservas = agregar_stockc152reservas.merge(fil_pedido_modif4,how='left', on='identificador')
modificar_stockc200xinventario = modificar_stockc152xinventario.merge(fil_inventario_modif4, how='left', on='identificador')

In [45]:
agregar_stockc200reservas.loc[agregar_stockc200reservas['trasl_ctd_C200_y'].isna(), 'trasl_ctd_C200_y'] = 0
agregar_stockc200reservas.loc[agregar_stockc200reservas['total_y'].isna(), 'total_y'] = agregar_stockc200reservas['total_x']

modificar_stockc200xinventario.loc[modificar_stockc200xinventario['total_y'].isna(), 'total_y'] = modificar_stockc200xinventario['total_x']

In [46]:
agregar_stockc200reservas['trasl_ctd_C200'] = agregar_stockc200reservas['trasl_ctd_C200_y']
agregar_stockc200reservas['total'] = agregar_stockc200reservas['total_y']

agregar_stockc200reservas.drop(columns=['trasl_ctd_C200_y','trasl_ctd_C200_x'], inplace=True)
agregar_stockc200reservas.drop(columns=['total_y','total_x'], inplace=True)


modificar_stockc200xinventario['total'] = modificar_stockc200xinventario['total_y']
modificar_stockc200xinventario.drop(columns=['total_x','total_y'], inplace=True)


In [47]:
agregar_stockc200reservas['diferencia'] = agregar_stockc200reservas['ctd_dif'] - agregar_stockc200reservas['total']
reservas_pendientes4 = agregar_stockc200reservas.loc[agregar_stockc200reservas['diferencia'] > 0].copy()

agregar_stockc200reservas.drop(columns='diferencia', inplace=True)
reservas_pendientes4.drop(columns='diferencia', inplace=True)

In [48]:
stock_restante4 = modificar_stockc200xinventario.loc[modificar_stockc200xinventario['total'] > 0].copy()

In [49]:
pedido_modif5, inventario_modif5 = _asignacion_centro_restante(reservas_pendientes4,stock_restante4,'C040')

In [50]:
fil_pedido_modif5 = pedido_modif5[['identificador','trasl_ctd_C040','total']]
fil_inventario_modif5 = inventario_modif5[['identificador','total']]

In [51]:
agregar_stockc040reservas = agregar_stockc200reservas.merge(fil_pedido_modif5,how='left', on='identificador')
modificar_stockc040xinventario = modificar_stockc200xinventario.merge(fil_inventario_modif5, how='left', on='identificador')

In [52]:
agregar_stockc040reservas.loc[agregar_stockc040reservas['trasl_ctd_C040_y'].isna(), 'trasl_ctd_C040_y'] = 0
agregar_stockc040reservas.loc[agregar_stockc040reservas['total_y'].isna(), 'total_y'] = agregar_stockc040reservas['total_x']

modificar_stockc040xinventario.loc[modificar_stockc040xinventario['total_y'].isna(), 'total_y'] = modificar_stockc040xinventario['total_x']

In [53]:
agregar_stockc040reservas['trasl_ctd_C040'] = agregar_stockc040reservas['trasl_ctd_C040_y']
agregar_stockc040reservas['total'] = agregar_stockc040reservas['total_y']

agregar_stockc040reservas.drop(columns=['trasl_ctd_C040_y','trasl_ctd_C040_x'], inplace=True)
agregar_stockc040reservas.drop(columns=['total_y','total_x'], inplace=True)


modificar_stockc040xinventario['total'] = modificar_stockc040xinventario['total_y']
modificar_stockc040xinventario.drop(columns=['total_x','total_y'], inplace=True)


In [54]:
agregar_stockc040reservas['diferencia'] = agregar_stockc040reservas['ctd_dif'] - agregar_stockc040reservas['total']
reservas_pendientes5 = agregar_stockc040reservas.loc[agregar_stockc040reservas['diferencia'] > 0].copy()

agregar_stockc040reservas.drop(columns='diferencia', inplace=True)
reservas_pendientes5.drop(columns='diferencia', inplace=True)

In [55]:
stock_restante5 = modificar_stockc040xinventario.loc[modificar_stockc040xinventario['total'] > 0].copy()

In [56]:
pedido_modif6, inventario_modif6 = _asignacion_centro_restante(reservas_pendientes5,stock_restante5,'C080')

In [57]:
fil_pedido_modif6 = pedido_modif6[['identificador','trasl_ctd_C080','total']]
fil_inventario_modif6 = pedido_modif6[['identificador','total']]

In [58]:
agregar_stockc080reservas = agregar_stockc040reservas.merge(fil_pedido_modif6,how='left', on='identificador')
modificar_stockc080xinventario = modificar_stockc040xinventario.merge(fil_inventario_modif6, how='left', on='identificador')

In [59]:
agregar_stockc080reservas.loc[agregar_stockc080reservas['trasl_ctd_C080_y'].isna(), 'trasl_ctd_C080_y'] = 0
agregar_stockc080reservas.loc[agregar_stockc080reservas['total_y'].isna(), 'total_y'] = agregar_stockc080reservas['total_x']

modificar_stockc080xinventario.loc[modificar_stockc080xinventario['total_y'].isna(), 'total_y'] = modificar_stockc080xinventario['total_x']

In [60]:
agregar_stockc080reservas['trasl_ctd_C080'] = agregar_stockc080reservas['trasl_ctd_C080_y']
agregar_stockc080reservas['total'] = agregar_stockc080reservas['total_y']

agregar_stockc080reservas.drop(columns=['trasl_ctd_C080_y','trasl_ctd_C080_x'], inplace=True)
agregar_stockc080reservas.drop(columns=['total_y','total_x'], inplace=True)


modificar_stockc080xinventario['total'] = modificar_stockc080xinventario['total_y']
modificar_stockc080xinventario.drop(columns=['total_x','total_y'], inplace=True)


In [61]:
agregar_stockc080reservas['diferencia'] = agregar_stockc080reservas['ctd_dif'] - agregar_stockc080reservas['total']
reservas_pendientes6 = agregar_stockc080reservas.loc[agregar_stockc080reservas['diferencia'] > 0].copy()

agregar_stockc080reservas.drop(columns='diferencia', inplace=True)
reservas_pendientes6.drop(columns='diferencia', inplace=True)

In [62]:
stock_restante6 = modificar_stockc080xinventario.loc[modificar_stockc080xinventario['total'] > 0].copy()

In [63]:
def _asign_pedidos_compra(reservas, pedidospendiente):
    # Ordenar los pedidos pendientes
    ped = pedidospendiente.sort_values(by=['fecha_entrega', 'identificador'], ascending=[True, True], na_position='last')

    # Proceso de asignación
    for i, reser in reservas.iterrows():
        cantidad_pedida = reser['ctd_dif'] - reser['total']
        compras_consolidadas = []
        fechas_consolidadas = []
        cantidades_consolidadas = []
        proveedor_consolidadas = []

        for j, pedi in ped.iterrows():
            if pedi['por_entr_ctd'] > 0:
                if pedi['por_entr_ctd'] >= cantidad_pedida:
                    compras_consolidadas.append(pedi['doc_compra'])
                    fechas_consolidadas.append(pedi['fecha_entrega'])
                    proveedor_consolidadas.append(pedi['nombre_proveedor'])
                    cantidades_consolidadas.append(cantidad_pedida)
                    ped.at[j, 'por_entr_ctd'] -= cantidad_pedida
                    cantidad_pedida = 0
                    break
                else:
                    compras_consolidadas.append(pedi['doc_compra'])
                    fechas_consolidadas.append(pedi['fecha_entrega'])
                    proveedor_consolidadas.append(pedi['nombre_proveedor'])
                    cantidades_consolidadas.append(pedi['por_entr_ctd'])
                    cantidad_pedida -= pedi['por_entr_ctd']
                    ped.at[j, 'por_entr_ctd'] = 0

        reservas.at[i, 'Ped_nro'] = ', '.join(map(str, compras_consolidadas))
        reservas.at[i, 'Ped_ctd'] = sum(cantidades_consolidadas)
        reservas.at[i, 'Ped_proveedor'] = ', '.join(map(str, proveedor_consolidadas))
        reservas.at[i, 'total'] = reser['total'] + sum(cantidades_consolidadas)
        if fechas_consolidadas:
            reservas.at[i, 'Ped_Fecha'] = fechas_consolidadas[-1]

    return reservas, ped

In [64]:
def asignacion_pedidoscompra(reservas, pedidoscompras):
    reserva = reservas.copy()
    pedidoscompra = pedidoscompras.copy()

    centros = pedidoscompra['centro'].unique()
    materiales = pedidoscompra['material'].unique()

    concatenar_reservas = []

    for centro in centros:
        for material in materiales:
            pedido_m = reserva[(reserva['centro'] == centro) & (reserva['material'] == material)]
            pedidos_filtrados = pedidoscompra[(pedidoscompra['centro'] == centro) & (pedidoscompra['material'] == material)]
            if not pedidos_filtrados.empty:
                _reservas, _pedidos = _asign_pedidos_compra(pedido_m, pedidos_filtrados)
                concatenar_reservas.append(_reservas)
                pedidoscompra.update(_pedidos)

    pedidos_consolidados = pd.concat(concatenar_reservas)

    return pedidos_consolidados, pedidoscompra

In [65]:
pedido_modif7,pedidoscompra_modif = asignacion_pedidoscompra(reservas_pendientes6, filtrar_cabezeras_estadoxpedidos)

In [66]:
fil_pedido_modif7 = pedido_modif7[['identificador','Ped_nro','Ped_Fecha','Ped_ctd','Ped_proveedor','total']]
fil_pedidoscompra =pedidoscompra_modif.loc[pedidoscompra_modif['por_entr_ctd'] >0]

In [67]:
agregar_pedidosxreservas = agregar_stockc080reservas.merge(fil_pedido_modif7,how='left', on='identificador')

In [68]:
agregar_pedidosxreservas.loc[agregar_pedidosxreservas['Ped_nro_y'].isna(), 'Ped_nro_y'] = agregar_pedidosxreservas['Ped_nro_x']
agregar_pedidosxreservas.loc[agregar_pedidosxreservas['Ped_Fecha_y'].isna(), 'Ped_Fecha_y'] = agregar_pedidosxreservas['Ped_Fecha_x']
agregar_pedidosxreservas.loc[agregar_pedidosxreservas['Ped_ctd_y'].isna(), 'Ped_ctd_y'] = agregar_pedidosxreservas['Ped_ctd_x']
agregar_pedidosxreservas.loc[agregar_pedidosxreservas['Ped_proveedor_y'].isna(), 'Ped_proveedor_y'] = agregar_pedidosxreservas['Ped_proveedor_x']
agregar_pedidosxreservas.loc[agregar_pedidosxreservas['total_y'].isna(), 'total_y'] = agregar_pedidosxreservas['total_x']

In [69]:
agregar_pedidosxreservas['Ped_nro'] = agregar_pedidosxreservas['Ped_nro_y']
agregar_pedidosxreservas['Ped_Fecha'] = agregar_pedidosxreservas['Ped_Fecha_y']
agregar_pedidosxreservas['Ped_ctd'] = agregar_pedidosxreservas['Ped_ctd_y']
agregar_pedidosxreservas['Ped_proveedor'] = agregar_pedidosxreservas['Ped_proveedor_y']
agregar_pedidosxreservas['total'] = agregar_pedidosxreservas['total_y']

agregar_pedidosxreservas.drop(columns=['Ped_nro_y','Ped_nro_x'], inplace=True)
agregar_pedidosxreservas.drop(columns=['Ped_Fecha_y','Ped_Fecha_x'], inplace=True)
agregar_pedidosxreservas.drop(columns=['Ped_ctd_y','Ped_ctd_x'], inplace=True)
agregar_pedidosxreservas.drop(columns=['Ped_proveedor_y','Ped_proveedor_x'], inplace=True)
agregar_pedidosxreservas.drop(columns=['total_y','total_x'], inplace=True)


In [70]:
agregar_pedidosxreservas['total_trasl'] = agregar_pedidosxreservas['trasl_ctd_C154'] + agregar_pedidosxreservas['trasl_ctd_C152'] +agregar_pedidosxreservas['trasl_ctd_C200'] + agregar_pedidosxreservas['trasl_ctd_C040'] + agregar_pedidosxreservas['trasl_ctd_C080']

In [71]:
fil_reservasxtrasl = agregar_pedidosxreservas.loc[(agregar_pedidosxreservas['total_trasl'] > 0)]

In [72]:
def _asign_pedidoscompraxtrasl(reservas, pedidospendiente):
    # Ordenar los pedidos pendientes
    ped = pedidospendiente.sort_values(by=['fecha_entrega', 'identificador'], ascending=[True, True], na_position='last')

    # Proceso de asignación
    for i, reser in reservas.iterrows():
        cantidad_pedida =  reser['total_trasl']
        compras_consolidadas = []
        fechas_consolidadas = []
        cantidades_consolidadas = []
        proveedor_consolidadas = []

        for j, pedi in ped.iterrows():
            if pedi['por_entr_ctd'] > 0:
                if pedi['por_entr_ctd'] >= cantidad_pedida:
                    compras_consolidadas.append(pedi['doc_compra'])
                    fechas_consolidadas.append(pedi['fecha_entrega'])
                    proveedor_consolidadas.append(pedi['nombre_proveedor'])
                    cantidades_consolidadas.append(cantidad_pedida)
                    ped.at[j, 'por_entr_ctd'] -= cantidad_pedida
                    cantidad_pedida = 0
                    break
                else:
                    compras_consolidadas.append(pedi['doc_compra'])
                    fechas_consolidadas.append(pedi['fecha_entrega'])
                    proveedor_consolidadas.append(pedi['nombre_proveedor'])
                    cantidades_consolidadas.append(pedi['por_entr_ctd'])
                    cantidad_pedida -= pedi['por_entr_ctd']
                    ped.at[j, 'por_entr_ctd'] = 0

        if pd.notna(reservas.at[i, 'Ped_nro']):
            reservas.at[i, 'Ped_nro'] = f"{reservas.at[i, 'Ped_nro']}, " + ', '.join(map(str, compras_consolidadas))
        else:
            reservas.at[i, 'Ped_nro'] = ', '.join(map(str, compras_consolidadas))

        reservas.at[i, 'Ped_ctd'] += sum(cantidades_consolidadas) if pd.notna(reservas.at[i, 'Ped_ctd']) else sum(cantidades_consolidadas)
        
        if pd.notna(reservas.at[i, 'Ped_proveedor']):
            reservas.at[i, 'Ped_proveedor'] = f"{reservas.at[i, 'Ped_proveedor']}, " + ', '.join(map(str, proveedor_consolidadas))
        else:
            reservas.at[i, 'Ped_proveedor'] = ', '.join(map(str, proveedor_consolidadas))
        
        if fechas_consolidadas:
            reservas.at[i, 'Ped_Fecha'] = fechas_consolidadas[-1]

    return reservas, ped

In [73]:
def asignacion_pedidoscompraxtrasl(reservas, pedidoscompras):
    reserva = reservas.copy()
    pedidoscompra = pedidoscompras.copy()

    centros = pedidoscompra['centro'].unique()
    materiales = pedidoscompra['material'].unique()

    concatenar_reservas = []

    for centro in centros:
        for material in materiales:
            pedido_m = reserva[(reserva['centro'] == centro) & (reserva['material'] == material)]
            pedidos_filtrados = pedidoscompra[(pedidoscompra['centro'] == centro) & (pedidoscompra['material'] == material)]
            if not pedidos_filtrados.empty:
                _reservas, _pedidos = _asign_pedidoscompraxtrasl(pedido_m, pedidos_filtrados)
                concatenar_reservas.append(_reservas)
                pedidoscompra.update(_pedidos)

    pedidos_consolidados = pd.concat(concatenar_reservas)

    return pedidos_consolidados, pedidoscompra

In [74]:
pedido_modif8,pedidoscompra_modif2 = asignacion_pedidoscompraxtrasl(fil_reservasxtrasl, fil_pedidoscompra)

In [75]:
fil_pedido_modif8 = pedido_modif8[['identificador','Ped_nro','Ped_Fecha','Ped_ctd','Ped_proveedor']]

In [76]:
agregar_pedidosxreservastrasl = agregar_pedidosxreservas.merge(fil_pedido_modif8,how='left', on='identificador')

In [77]:
agregar_pedidosxreservastrasl.loc[agregar_pedidosxreservastrasl['Ped_nro_y'].isna(), 'Ped_nro_y'] = agregar_pedidosxreservastrasl['Ped_nro_x']
agregar_pedidosxreservastrasl.loc[agregar_pedidosxreservastrasl['Ped_Fecha_y'].isna(), 'Ped_Fecha_y'] = agregar_pedidosxreservastrasl['Ped_Fecha_x']
agregar_pedidosxreservastrasl.loc[agregar_pedidosxreservastrasl['Ped_ctd_y'].isna(), 'Ped_ctd_y'] = agregar_pedidosxreservastrasl['Ped_ctd_x']
agregar_pedidosxreservastrasl.loc[agregar_pedidosxreservastrasl['Ped_proveedor_y'].isna(), 'Ped_proveedor_y'] = agregar_pedidosxreservastrasl['Ped_proveedor_x']

In [78]:
agregar_pedidosxreservastrasl['Ped_nro'] = agregar_pedidosxreservastrasl['Ped_nro_y']
agregar_pedidosxreservastrasl['Ped_Fecha'] = agregar_pedidosxreservastrasl['Ped_Fecha_y']
agregar_pedidosxreservastrasl['Ped_ctd'] = agregar_pedidosxreservastrasl['Ped_ctd_y']
agregar_pedidosxreservastrasl['Ped_proveedor'] = agregar_pedidosxreservastrasl['Ped_proveedor_y']

agregar_pedidosxreservastrasl.drop(columns=['Ped_nro_y','Ped_nro_x'], inplace=True)
agregar_pedidosxreservastrasl.drop(columns=['Ped_Fecha_y','Ped_Fecha_x'], inplace=True)
agregar_pedidosxreservastrasl.drop(columns=['Ped_ctd_y','Ped_ctd_x'], inplace=True)
agregar_pedidosxreservastrasl.drop(columns=['Ped_proveedor_y','Ped_proveedor_x'], inplace=True)


In [79]:
fil_reserv = agregar_pedidosxreservastrasl[['identificador','stock_asignado','trasl_ctd_C154','trasl_ctd_C152','trasl_ctd_C200','trasl_ctd_C040','trasl_ctd_C080','Ped_nro','Ped_Fecha','Ped_ctd','Ped_proveedor','total','total_trasl']]

In [80]:
abastecimiento_reservas = reservas.merge(fil_reserv, how='left', on='identificador')

In [81]:
abastecimiento_reservas.loc[abastecimiento_reservas['stock_asignado_y'].isna(), 'stock_asignado_y'] = abastecimiento_reservas['stock_asignado_x']
abastecimiento_reservas.loc[abastecimiento_reservas['trasl_ctd_C154_y'].isna(), 'trasl_ctd_C154_y'] = abastecimiento_reservas['trasl_ctd_C154_x']
abastecimiento_reservas.loc[abastecimiento_reservas['trasl_ctd_C152_y'].isna(), 'trasl_ctd_C152_y'] = abastecimiento_reservas['trasl_ctd_C152_x']
abastecimiento_reservas.loc[abastecimiento_reservas['trasl_ctd_C200_y'].isna(), 'trasl_ctd_C200_y'] = abastecimiento_reservas['trasl_ctd_C200_x']
abastecimiento_reservas.loc[abastecimiento_reservas['trasl_ctd_C040_y'].isna(), 'trasl_ctd_C040_y'] = abastecimiento_reservas['trasl_ctd_C040_x']
abastecimiento_reservas.loc[abastecimiento_reservas['trasl_ctd_C080_y'].isna(), 'trasl_ctd_C080_y'] = abastecimiento_reservas['trasl_ctd_C080_x']
abastecimiento_reservas.loc[abastecimiento_reservas['Ped_nro_y'].isna(), 'Ped_nro_y'] = abastecimiento_reservas['Ped_nro_x']
abastecimiento_reservas.loc[abastecimiento_reservas['Ped_Fecha_y'].isna(), 'Ped_Fecha_y'] = abastecimiento_reservas['Ped_Fecha_x']
abastecimiento_reservas.loc[abastecimiento_reservas['Ped_ctd_y'].isna(), 'Ped_ctd_y'] = abastecimiento_reservas['Ped_ctd_x']
abastecimiento_reservas.loc[abastecimiento_reservas['Ped_proveedor_y'].isna(), 'Ped_proveedor_y'] = abastecimiento_reservas['Ped_proveedor_x']
abastecimiento_reservas.loc[abastecimiento_reservas['total'].isna(), 'total'] = 0
abastecimiento_reservas.loc[abastecimiento_reservas['total_trasl'].isna(), 'total_trasl'] = 0


In [82]:
abastecimiento_reservas['stock_asignado'] = abastecimiento_reservas['stock_asignado_y']
abastecimiento_reservas['trasl_ctd_C154'] = abastecimiento_reservas['trasl_ctd_C154_y']
abastecimiento_reservas['trasl_ctd_C152'] = abastecimiento_reservas['trasl_ctd_C152_y']
abastecimiento_reservas['trasl_ctd_C200'] = abastecimiento_reservas['trasl_ctd_C200_y']
abastecimiento_reservas['trasl_ctd_C040'] = abastecimiento_reservas['trasl_ctd_C040_y']
abastecimiento_reservas['trasl_ctd_C080'] = abastecimiento_reservas['trasl_ctd_C080_y']

abastecimiento_reservas['Ped_nro'] = abastecimiento_reservas['Ped_nro_y']
abastecimiento_reservas['Ped_Fecha'] = abastecimiento_reservas['Ped_Fecha_y']
abastecimiento_reservas['Ped_ctd'] = abastecimiento_reservas['Ped_ctd_y']
abastecimiento_reservas['Ped_proveedor'] = abastecimiento_reservas['Ped_proveedor_y']

abastecimiento_reservas.drop(columns=['stock_asignado_y','stock_asignado_x'], inplace=True)
abastecimiento_reservas.drop(columns=['trasl_ctd_C154_y','trasl_ctd_C154_x'], inplace=True)
abastecimiento_reservas.drop(columns=['trasl_ctd_C152_y','trasl_ctd_C152_x'], inplace=True)
abastecimiento_reservas.drop(columns=['trasl_ctd_C200_y','trasl_ctd_C200_x'], inplace=True)
abastecimiento_reservas.drop(columns=['trasl_ctd_C040_y','trasl_ctd_C040_x'], inplace=True)
abastecimiento_reservas.drop(columns=['trasl_ctd_C080_y','trasl_ctd_C080_x'], inplace=True)

abastecimiento_reservas.drop(columns=['Ped_nro_y','Ped_nro_x'], inplace=True)
abastecimiento_reservas.drop(columns=['Ped_Fecha_y','Ped_Fecha_x'], inplace=True)
abastecimiento_reservas.drop(columns=['Ped_ctd_y','Ped_ctd_x'], inplace=True)
abastecimiento_reservas.drop(columns=['Ped_proveedor_y','Ped_proveedor_x'], inplace=True)

In [83]:
abastecimiento_reservas['Q_pendiente'] = abastecimiento_reservas['ctd_dif'] - abastecimiento_reservas['total'] 

In [84]:
abastecimiento_reservas.loc[abastecimiento_reservas['total_trasl']>0,'trasl_fecha'] = (datetime.now() + timedelta(days=2)).strftime('%Y-%m-%d')

In [85]:
abastecimiento_reservas['Est_material'] = ''
abastecimiento_reservas['Est_abastecimiento'] = ''
abastecimiento_reservas.loc[abastecimiento_reservas['Q_pendiente'] == 0,'Est_material'] = 'Completo'
abastecimiento_reservas.loc[abastecimiento_reservas['Q_pendiente'] > 0,'Est_material'] = 'Faltante'
abastecimiento_reservas.loc[(abastecimiento_reservas['Q_pendiente'] > 0) & ((abastecimiento_reservas['ctd_dif'] - abastecimiento_reservas['Q_pendiente']) != 0),'Est_material'] = 'Parcial'

abastecimiento_reservas.loc[abastecimiento_reservas['Q_pendiente'] == 0,'Est_abastecimiento'] = 'Abastecido'
abastecimiento_reservas.loc[abastecimiento_reservas['Q_pendiente'] != 0,'Est_abastecimiento'] = 'No_Abastecido'

In [86]:
abastecimiento_reservas['Est_Compra'] = ''
abastecimiento_reservas.loc[((abastecimiento_reservas['ctd_dif'] - abastecimiento_reservas['stock_asignado']) ==0),'Est_Compra'] = 'Total_abs_inv'
abastecimiento_reservas.loc[( abastecimiento_reservas['total_trasl'] >0),'Est_Compra'] = 'Segmien_Traslado'
abastecimiento_reservas.loc[( abastecimiento_reservas['Ped_ctd'] >0),'Est_Compra'] = 'Segmien_Oc'
abastecimiento_reservas.loc[abastecimiento_reservas['Q_pendiente'] >0,'Est_Compra'] = 'Por_Pedir'

In [87]:
abastecimiento_reservas['tipo_Ot'] = abastecimiento_reservas['orden_texto'].str[:3]
abastecimiento_reservas['semana_Ot_txt'] = abastecimiento_reservas['orden_revision_texto'].str[-2:]

abastecimiento_reservas['fe_nece'] = pd.to_datetime(abastecimiento_reservas['fe_nece'])
abastecimiento_reservas['felib'] = pd.to_datetime(abastecimiento_reservas['felib'])

abastecimiento_reservas['semana_Ot'] = abastecimiento_reservas['fe_nece'].dt.isocalendar().week
abastecimiento_reservas['semana_lib'] = abastecimiento_reservas['felib'].dt.isocalendar().week


In [88]:
abast_reservas = abastecimiento_reservas.merge(tbl_flota, how='left', on='equipo')

In [89]:
abast_reservas['fechahora'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


In [90]:
cargar_abastecimiento = abast_reservas.reindex(['index','fechahora','identificador','orden','felib','aviso','empldeno','zona','equipo','linegnombre','modelo','eqdescrip','repercusion','cliente','proyecto','ave_ini_fecha','ave_fin_fecha','parada',
                                        'parada_dias','ubitec_deno','local','undneg','centro_deno','eq_est_usu_deno','flota_tipo','fe_despacho','eq_serie_motor','eq_serie_equipo','eq_est_ot_deno','clase','grupo_planif',
                                        'cliente_nro','proyecto_nro','supres_nombre','criticidad_deno','eqvaloradq_usd','repercusiondeno','oden_clase_deno','coti_esta_deno','avisodespacho','estatususuarioorden',
                                        'aviso_fecreado','orden_fe_fin_ext','orden_fe_ini_ext','fabricaequipo','orden_fecietec','orden_claact_deno','gruplan_deno','centro_y','eq_clase','orden_tipimput','orden_fecrea','orden_texto',
                                        'orden_prio','prio_orden','prio_orden_texto','ot_abierto','ot_liberado','ot_cierrete','ot_cerrado','hruta_contador','hruta_grupo','aviso_prioridad_deno','aviso_clase_deno','orden_revision','orden_revision_texto','orden_causas_cod',
                                        'orden_causas','orden_causas_texto','cotizacion','pedido_venta','orden_pos_mtto','plan_mantenimiento','fechacarga_y','reserva','reservapos','material','materialdeno','ctd_nec','ctd_reduc',
                                        'ctd_dif','um_base','usuario','centro_x','almacen','lote','fe_nece','orden_op','dest_merc','imputacion','ce_coste','salida_fin','relevplnec','grafo','elem_pep','febase','reservaestatus','reservaimputacion',
                                        'indicador_dh','reserva_pos_borrado','reserva_mov_permit','ate_accion','ate_codigosap','ate_cantidad','ate_UMB','ate_orden','ate_fecha','valor_flota','reservasolped','detalle01','detalle02',
                                        'stock_asignado','trasl_ctd_C154','trasl_ctd_C040','trasl_ctd_C080','trasl_ctd_C200','trasl_ctd_C152','trasl_fecha','Ped_nro','Ped_Fecha','Ped_ctd','Ped_proveedor','estadoreserva','Cod_mate_antiguo','Grp_mat','Jerarquia_produc','total_trasl',
                                        'Q_pendiente','total','Est_material','Est_abastecimiento','Est_Compra','tipo_Ot','semana_Ot_txt','semana_Ot','semana_lib'],axis=1)

In [91]:
cabezera_abastecimiento = ['id','fechacarga','identificador','orden','felib','aviso','empldeno','zona','equipo','linea_negocio','modelo','eqdescrip','repercusion','cliente','proyecto','ave_ini_fecha','ave_fin_fecha','parada','parada_dias','ubitec_deno',
                            'local','undneg','centro_deno','eq_est_usu_deno','flota_tipo','fe_despacho','eq_serie_motor','eq_serie_equipo','eq_est_ot_deno','clase','grupo_planif','cliente_nro','proyecto_nro','supres_nombre',
                            'criticidad_deno','eqvaloradq_usd','repercusiondeno','oden_clase_deno','coti_esta_deno','avisodespacho','estatususuarioorden','aviso_fecreado','orden_fe_fin_ext','orden_fe_ini_ext','fabricaequipo',
                            'orden_fecietec','orden_claact_deno','gruplan_deno','centro','eq_clase','orden_tipimput','orden_fecrea','orden_texto','orden_prio','prio_orden','prio_orden_texto','ot_abierto','ot_liberado','ot_cierrete','ot_cerrado','hruta_contador',
                            'hruta_grupo','aviso_prioridad_deno','aviso_clase_deno','orden_revision','orden_revision_texto','orden_causas_cod','orden_causas','orden_causas_texto','cotizacion','pedido_venta','orden_pos_mtto','plan_mantenimiento',
                            'reserva_fechacarga','reserva','reservapos','material','materialdeno','ctd_nec','ctd_reduc','ctd_dif','um_base','usuario','reserva_centro','reserva_almacen','lote','fe_nece','orden_op','dest_merc','imputacion','ce_coste',
                            'salida_fin','relevplnec','grafo','elem_pep','febase','reservaestatus','reservaimputacion','indicador_dh','reserva_pos_borrado','reserva_mov_permit','ate_accion','ate_codigosap','ate_cantidad','ate_UMB','ate_orden',
                            'ate_fecha','valor_flota','reservasolped','detalle01','detalle02','stock_asignado','trasl_ctd_c154','trasl_ctd_c040','trasl_ctd_c080','trasl_ctd_c200','trasl_ctd_c152','trasl_fecha','Ped_nro','Ped_Fecha','Ped_ctd','Ped_proveedor','estadoreserva',
                            'Cod_mat_antig','Grp_material','Jerarquia','total_trasl','Q_pendiente','Q_asignado','Est_material','Est_abastecimiento','Estado_Compra','tipo_Ot','semana_Ot_txt','semana_Ot','semana_lib']

In [ ]:
cargar_abastecimientosin_diesel = cargar_abastecimiento[cargar_abastecimiento['material'] != 'DIESEL'].copy()

In [ ]:
connection = crear_conexion()
consult_data(connection,"truncate table tbl_abastecimiento")
insert_data(connection,'tbl_abastecimiento',cargar_abastecimientosin_diesel,cabezera_abastecimiento,1000)

Conexion Exitosa
Se insertaron: 10730 registros. en la tabla tbl_abastecimiento


In [ ]:
connection = crear_conexion()
insert_data(connection,'tbl_abastecimiento_hist',cargar_abastecimientosin_diesel,cabezera_abastecimiento,1000)

In [93]:
len(cargar_abastecimiento.loc[cargar_abastecimiento['ate_accion']=='FIN'])  

307